**Ajuste de Hiperparametros usando XGBooster**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/TrabalhoTEI

import pandas as pd
import numpy as np  # Importar numpy para manipulação de arrays
import optuna
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Carregar os dados
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Criar LabelEncoders para cada coluna categórica
label_encoders = {}

# Transformação no conjunto de treino
for col in train_df.columns:
    if col != 'id':  # Não transformamos a coluna 'id'
        le = LabelEncoder()
        train_df[col] = le.fit_transform(train_df[col])
        label_encoders[col] = le  # Armazenar o encoder para usar no teste

# Transformação no conjunto de teste
for col in test_df.columns:
    if col != 'id':  # Não transformamos a coluna 'id'
        le = label_encoders.get(col)  # Recuperar o encoder correspondente
        if le is not None:
            # Identificar e tratar rótulos desconhecidos
            unknown_labels = set(test_df[col].unique()) - set(le.classes_)
            if unknown_labels:
                le.classes_ = np.append(le.classes_, list(unknown_labels))
            test_df[col] = le.transform(test_df[col])

# Preparar dados para treinamento
X = train_df.drop(['id', 'class'], axis=1)
y = train_df['class']

# Definir a função de objetivo para o Optuna
def objective(trial):
    # Definindo os hiperparâmetros a serem otimizados
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }

    model = xgb.XGBClassifier(**param, random_state=42)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Ajustar o modelo
    model.fit(X_train, y_train)

    # Prever e calcular o MCC
    y_pred = model.predict(X_val)
    mcc = matthews_corrcoef(y_val, y_pred)

    return mcc

# Criar e otimizar o estudo com Optuna
study = optuna.create_study(direction='maximize')  # Maximizar o MCC
study.optimize(objective, n_trials=50)  # Número de combinações a serem testadas

# Resultados
print('Melhores hiperparâmetros: ', study.best_params)
print('Melhor MCC: ', study.best_value)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/TrabalhoTEI


[I 2024-09-08 17:30:53,641] A new study created in memory with name: no-name-c18b6af8-bc13-4fe6-843b-65b651de87f9
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:31:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-08 17:31:43,622] Trial 0 finished with value: 0.9836063756022954 and parameters: {'max_depth': 9, 'subsample': 0.9543988826106495, 'colsample_bytree': 0.934064691238299}. Best is trial 0 with value: 0.9836063756022954.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:31:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-08 17:32:32,941] Trial 1 finished with value: 0.9835743160642563 and parameters: {'max_depth': 9, 'subsample': 0.8844483649335647, 'colsample_bytree': 0.7688215271075949}. Best is trial 0 with value: 0.9836063756022954.
/

Melhores hiperparâmetros:  {'max_depth': 9, 'subsample': 0.9276592101046933, 'colsample_bytree': 0.9119456082162385}
Melhor MCC:  0.9837397566419347


Melhores hiperparâmetros:  {'max_depth': 9, 'subsample': 0.9276592101046933, 'colsample_bytree': 0.9119456082162385}  
Melhor MCC:  0.9837397566419347

**Testando o melhor hiperparametro achado**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/rdc/tei

import pandas as pd
import numpy as np  # Importar numpy para manipulação de arrays
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Carregar os dados
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Criar LabelEncoders para cada coluna categórica
label_encoders = {}

# Transformação no conjunto de treino
for col in train_df.columns:
    if col != 'id':  # Não transformamos a coluna 'id'
        le = LabelEncoder()
        train_df[col] = le.fit_transform(train_df[col])
        label_encoders[col] = le  # Armazenar o encoder para usar no teste

# Transformação no conjunto de teste
for col in test_df.columns:
    if col != 'id':  # Não transformamos a coluna 'id'
        le = label_encoders.get(col)  # Recuperar o encoder correspondente
        if le is not None:
            # Identificar e tratar rótulos desconhecidos
            unknown_labels = set(test_df[col].unique()) - set(le.classes_)
            if unknown_labels:
                le.classes_ = np.append(le.classes_, list(unknown_labels))
            test_df[col] = le.transform(test_df[col])

# Preparar dados para treinamento
X = train_df.drop(['id', 'class'], axis=1)
y = train_df['class']

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/rdc/tei


In [ ]:
best_params = {
    'max_depth': 9,
    'subsample': 0.9276592101046933,
    'colsample_bytree': 0.9119456082162385,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'random_state': 42
}

model = xgb.XGBClassifier(**best_params)
model.fit(X, y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:29:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9119456082162385, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
test_predictions = model.predict(test_df.drop('id', axis=1))


**Score Kaggle: 0.98236**

In [ ]:
test_df['class'] = test_predictions
test_df['class'].replace({0: 'e', 1: 'p'}, inplace=True)
test_df[["id","class"]].to_csv("p_col.csv", index=False)

In [ ]:
study = optuna.create_study(direction='maximize')  # Maximizar o MCC
study.optimize(objective, n_trials=50)  # Número de combinações a serem testadas

[I 2024-09-08 18:18:01,202] A new study created in memory with name: no-name-1e7e7289-7f69-4c73-9a85-49a1b7d8f514
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:18:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-08 18:18:46,514] Trial 0 finished with value: 0.983025190756654 and parameters: {'max_depth': 7, 'subsample': 0.7368740958366682, 'colsample_bytree': 0.8223889157361134}. Best is trial 0 with value: 0.983025190756654.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:18:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-08 18:19:31,378] Trial 1 finished with value: 0.9833160015663082 and parameters: {'max_depth': 8, 'subsample': 0.7882948127569208, 'colsample_bytree': 0.9055085905606409}. Best is trial 1 with value: 0.9833160015663082.
/u

In [ ]:
print('Melhores hiperparâmetros: ', study.best_params)
print('Melhor MCC: ', study.best_value)